# 🎯 LIDAR Grasp Detection v12

## ใหม่ใน v12
| Feature | Description |
|---------|-------------|
| **📐 LIDAR Offset Calibration** | Calibrate ความสัมพันธ์ LIDAR-Gripper 1 ครั้ง |
| **🔄 R_OFFSET First** | หมุนไปที่ R_OFFSET ก่อนวัด (ป้องกัน LIDAR เปลี่ยนตำแหน่ง) |
| **📏 Move LIDAR to Object** | ย้าย LIDAR ไปตรงวัตถุก่อนวัด |

## 1️⃣ Imports

In [1]:
import cv2
import numpy as np
import time
import socket
import serial
from ultralytics import YOLO

print("✓ Imports")

✓ Imports


## 2️⃣ Hardware Configuration

In [2]:
ROBOT_IP = '192.168.1.6'
ESP32_PORT = 'COM9'
ESP32_BAUDRATE = 115200
CAMERA_ID = 2

HOMOGRAPHY_MATRIX = np.load('homography_matrix.npy')

print("✓ Hardware config")

✓ Hardware config


## 3️⃣ Configuration

In [4]:
# === CALIBRATED VALUES ===
PIXELS_PER_MM = 2.7703

# === Robot R Rotation ===
ROBOT_R_OFFSET = -25.55  # R ที่ใช้ตอน calibrate และวัด LIDAR

# === Z Heights ===
Z_FLOOR = -64
#Z_MEASURE = 201  # ความสูงที่ LIDAR วัดได้ดี (>200mm จากพื้น)
Z_MEASURE = 230  # ความสูงที่ LIDAR วัดได้ดี (>200mm จากพื้น)


# === LIDAR Configuration ===
LIDAR_Z_OFFSET = 39  # mm (ระยะแนวตั้ง จาก LIDAR ถึงปลาย gripper)

# === v12: LIDAR XY Offset (จาก Calibration) ===
# เมื่อ Robot อยู่ที่ R=ROBOT_R_OFFSET:
# - LIDAR_X_OFFSET = ระยะ X จากกลาง gripper ถึง LIDAR
# - LIDAR_Y_OFFSET = ระยะ Y จากกลาง gripper ถึง LIDAR
LIDAR_X_OFFSET = 25.08   # mm (จะได้จาก calibration)
LIDAR_Y_OFFSET = 20.71   # mm (จะได้จาก calibration)

# === Drop Position ===
DROP_POS = (-253.07, 115.17, -17.07, -62.78)

# === Gripper ===
GRIPPER_MAX_WIDTH_MM = 54
GRIPPER_OPEN_MARGIN_MM = 5
GRIPPER_GRIP_MARGIN_MM = 5

# === Detection ===
MIN_OBJECT_AREA = 1000
#YOLO_CONFIDENCE = 0.25
YOLO_CONFIDENCE = 0.15

print("✓ Configuration (v12 with LIDAR Offset Calibration)")

✓ Configuration (v12 with LIDAR Offset Calibration)


## 4️⃣ Load Models

In [5]:
print("Loading YOLOv8...")
yolo_model = YOLO('yolov8n.pt')
print("✅ YOLO loaded")

Loading YOLOv8...
✅ YOLO loaded


## 5️⃣ Classes Definition

In [6]:
class SmartGripperController:
    """Gripper + LIDAR Controller (v12)"""
    CALIB_ANGLES = np.array([22, 30, 40, 50, 60, 70, 80, 90, 96])
    CALIB_WIDTHS = np.array([54.0, 52.0, 48.0, 40.0, 32.0, 23.0, 12.0, 3.0, 0.0])
    
    def __init__(self, port='COM9', baudrate=115200):
        self.port = port
        self.baudrate = baudrate
        self.serial = None
        self.target_width = None
        
    def connect(self):
        try:
            self.serial = serial.Serial(self.port, self.baudrate, timeout=2)
            time.sleep(2)
            self.serial.reset_input_buffer()
            print(f"✅ Gripper+LIDAR on {self.port}")
            return True
        except Exception as e:
            print(f"❌ {e}")
            return False
    
    def disconnect(self):
        if self.serial: 
            self.serial.close()
            self.serial = None
    
    def send_command(self, cmd):
        if self.serial and self.serial.is_open:
            self.serial.reset_input_buffer()
            self.serial.write((cmd + '\n').encode())
            time.sleep(0.3)
    
    def mm_to_angle(self, width_mm):
        width = max(0.0, min(54.0, width_mm))
        return int(round(np.interp(width, self.CALIB_WIDTHS[::-1], self.CALIB_ANGLES[::-1])))
    
    def open_for_object(self, width_mm):
        self.target_width = width_mm
        open_w = min(54.0, width_mm + GRIPPER_OPEN_MARGIN_MM)
        angle = self.mm_to_angle(open_w)
        print(f"🦾 Open: {open_w:.1f}mm ({angle}°)")
        self.send_command(f'G{angle}')
    
    def grip_object(self, width_mm):
        grip_w = max(0.0, width_mm - GRIPPER_GRIP_MARGIN_MM)
        angle = self.mm_to_angle(grip_w)
        print(f"🦾 Grip: {grip_w:.1f}mm ({angle}°)")
        self.send_command(f'G{angle}')
    
    def release(self):
        open_w = min(54.0, (self.target_width or 30) + 10)
        self.send_command(f'G{self.mm_to_angle(open_w)}')
        self.target_width = None
    
    def read_lidar(self, samples=3):
        """Read LIDAR distance in mm"""
        if not self.serial or not self.serial.is_open:
            return None
        
        readings = []
        for _ in range(samples):
            self.serial.reset_input_buffer()
            self.serial.write(b'L\n')
            
            start = time.time()
            while time.time() - start < 1.0:
                if self.serial.in_waiting > 0:
                    response = self.serial.readline().decode().strip()
                    if response.startswith("LIDAR:") and "ERR" not in response:
                        try:
                            dist = int(response.split(":")[1])
                            readings.append(dist)
                        except:
                            pass
                        break
            time.sleep(0.05)
        
        if readings:
            return int(np.median(readings))
        return None

print("✓ Gripper+LIDAR class")

✓ Gripper+LIDAR class


In [7]:
class DobotControllerTCP:
    def __init__(self, homography_matrix=None, r_offset=-25.55):
        self.dashboard_port = 29999
        self.sock = None
        self.H = homography_matrix
        self.r_offset = r_offset
        
    def connect(self, ip):
        try:
            self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.sock.settimeout(5)
            self.sock.connect((ip, self.dashboard_port))
            self.send_command("ClearError()")
            time.sleep(0.5)
            self.send_command("EnableRobot()")
            time.sleep(4)
            self.send_command("User(1)")
            self.send_command("Tool(1)")
            self.send_command("SpeedFactor(50)")
            print("✅ Robot connected!")
            return True
        except Exception as e:
            print(f"Error: {e}")
            return False

    def send_command(self, cmd):
        if self.sock:
            self.sock.send((cmd + "\n").encode("utf-8"))
            return self.sock.recv(1024).decode("utf-8")

    def home(self):
        print("🤖 HOME...")
        self.send_command("MovJ(-253.07, 115.17, -17.07, -62.78)")
        time.sleep(4)

    def move_to(self, x, y, z, r=0):
        cmd = f"MovJ({x},{y},{z},{r})"
        print(f"   → {cmd}")
        return self.send_command(cmd)
    
    def move_to_and_wait(self, x, y, z, r=0, wait=3):
        self.move_to(x, y, z, r)
        time.sleep(wait)
    
    def joint_move(self, j1=0, j2=0, j3=0, j4=0):
        cmd = f"JointMovJ({j1},{j2},{j3},{j4})"
        print(f"   → {cmd}")
        return self.send_command(cmd)
    
    def joint_move_and_wait(self, j1=0, j2=0, j3=0, j4=0, wait=3):
        self.joint_move(j1, j2, j3, j4)
        time.sleep(wait)

    def pixel_to_robot(self, u, v):
        if self.H is None: return None, None
        pt = np.array([u, v, 1], dtype=np.float32)
        res = np.dot(self.H, pt)
        return res[0]/res[2], res[1]/res[2]
    
    def camera_angle_to_robot_r(self, camera_angle):
        return self.r_offset - camera_angle

print("✓ Robot class")

✓ Robot class


In [ ]:
"""class ObjectDetector:
    def __init__(self, yolo_model, pixels_per_mm):
        self.yolo = yolo_model
        self.ppm = pixels_per_mm
    
    def detect(self, frame):
        objects = []
        results = self.yolo(frame, conf=YOLO_CONFIDENCE, verbose=False)
        
        for r in results:
            for box in r.boxes:
                x1,y1,x2,y2 = map(int, box.xyxy[0])
                conf = float(box.conf[0])
                
                roi = frame[y1:y2, x1:x2]
                if roi.size == 0: continue
                
                gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
                _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
                contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                
                if contours:
                    cnt = max(contours, key=cv2.contourArea)
                    cnt = cnt + np.array([x1, y1])
                    rect = cv2.minAreaRect(cnt)
                    cx, cy = int(rect[0][0]), int(rect[0][1])
                    
                    objects.append({
                        'bbox': (x1, y1, x2-x1, y2-y1),
                        'center': (cx, cy),
                        'rect': rect,
                        'contour': cnt,
                        'conf': conf,
                        'area': cv2.contourArea(cnt)
                    })
        
        return objects

print("✓ Object Detector class")"""
# ObjectDetector v12.1 - with Color Detection Fallback
# Copy cell นี้ไปแทนที่ ObjectDetector เดิม

class ObjectDetector:
    """v12.1: YOLO + Color Detection Fallback"""
    
    def __init__(self, yolo_model, pixels_per_mm):
        self.yolo = yolo_model
        self.ppm = pixels_per_mm
        
        # Color ranges for detection (HSV)
        self.color_ranges = {
            'blue': ([100, 100, 50], [130, 255, 255]),
            'green': ([40, 50, 50], [80, 255, 255]),
            'red1': ([0, 100, 50], [10, 255, 255]),
            'red2': ([170, 100, 50], [180, 255, 255]),
            'yellow': ([20, 100, 100], [35, 255, 255]),
        }
    
    def detect(self, frame):
        """Detect objects using YOLO first, then color fallback"""
        # Try YOLO first
        objects = self._detect_yolo(frame)
        
        # If YOLO finds nothing, use color detection
        if len(objects) == 0:
            objects = self._detect_color(frame)
        
        return objects
    
    def _detect_yolo(self, frame):
        """YOLO-based detection"""
        objects = []
        results = self.yolo(frame, conf=YOLO_CONFIDENCE, verbose=False)
        
        for r in results:
            for box in r.boxes:
                cls_id = int(box.cls[0])
                # Skip person class (id=0)
                if cls_id == 0:
                    continue
                    
                x1,y1,x2,y2 = map(int, box.xyxy[0])
                conf = float(box.conf[0])
                
                roi = frame[y1:y2, x1:x2]
                if roi.size == 0: continue
                
                gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
                _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
                contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                
                if contours:
                    cnt = max(contours, key=cv2.contourArea)
                    cnt = cnt + np.array([x1, y1])
                    rect = cv2.minAreaRect(cnt)
                    cx, cy = int(rect[0][0]), int(rect[0][1])
                    
                    objects.append({
                        'bbox': (x1, y1, x2-x1, y2-y1),
                        'center': (cx, cy),
                        'rect': rect,
                        'contour': cnt,
                        'conf': conf,
                        'area': cv2.contourArea(cnt),
                        'method': 'YOLO'
                    })
        
        return objects
    
    def _detect_color(self, frame):
        """Color-based detection fallback"""
        objects = []
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        # Combined mask for all colors
        combined_mask = np.zeros(hsv.shape[:2], dtype=np.uint8)
        
        for color_name, (lower, upper) in self.color_ranges.items():
            lower = np.array(lower)
            upper = np.array(upper)
            mask = cv2.inRange(hsv, lower, upper)
            combined_mask = cv2.bitwise_or(combined_mask, mask)
        
        # Clean up mask
        kernel = np.ones((5,5), np.uint8)
        combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_OPEN, kernel)
        combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_CLOSE, kernel)
        
        # Find contours
        contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area < MIN_OBJECT_AREA:
                continue
            
            # Get bounding box and rect
            x, y, w, h = cv2.boundingRect(cnt)
            rect = cv2.minAreaRect(cnt)
            cx, cy = int(rect[0][0]), int(rect[0][1])
            
            objects.append({
                'bbox': (x, y, w, h),
                'center': (cx, cy),
                'rect': rect,
                'contour': cnt,
                'conf': 0.8,
                'area': area,
                'method': 'Color'
            })
        
        # Sort by area (largest first)
        objects = sorted(objects, key=lambda o: o['area'], reverse=True)
        
        return objects

print("✅ ObjectDetector v12.1 (with Color Detection)")


✓ ObjectDetectorV13 (Saturation + Edge - All Colors)


In [17]:
class PCAGraspSelector:
    def __init__(self, pixels_per_mm):
        self.ppm = pixels_per_mm
    
    def analyze_object(self, obj):
        cnt = obj.get('contour')
        if cnt is None or len(cnt) < 5:
            return self._fallback(obj)
        
        pts = cnt.reshape(-1, 2).astype(np.float64)
        mean = np.mean(pts, axis=0)
        pts_centered = pts - mean
        
        cov = np.cov(pts_centered.T)
        eigenvalues, eigenvectors = np.linalg.eig(cov)
        idx = np.argsort(eigenvalues)[::-1]
        eigenvectors = eigenvectors[:, idx]
        
        major = eigenvectors[:, 0]
        minor = eigenvectors[:, 1]
        
        angle = np.degrees(np.arctan2(major[1], major[0]))
        
        proj = np.dot(pts_centered, minor)
        width_mm = (np.max(proj) - np.min(proj)) / self.ppm
        
        cx, cy = int(mean[0]), int(mean[1])
        grasp_angle = self._normalize(angle + 90)
        
        grasps = []
        if width_mm <= GRIPPER_MAX_WIDTH_MM:
            grasps.append({
                'center': (cx, cy),
                'width_mm': width_mm,
                'camera_angle': grasp_angle,
                'score': 1.0,
                'type': 'PCA'
            })
        return grasps if grasps else self._fallback(obj)
    
    def _fallback(self, obj):
        rect = obj.get('rect')
        if not rect: return []
        (cx, cy), (w, h), angle = rect
        grip_w = min(w, h) / self.ppm
        grip_a = angle + 90 if w < h else angle
        if grip_w <= GRIPPER_MAX_WIDTH_MM:
            return [{'center': (int(cx), int(cy)), 'width_mm': grip_w, 
                     'camera_angle': self._normalize(grip_a), 'score': 0.6, 'type': 'Rect'}]
        return []
    
    def _normalize(self, a):
        while a > 90: a -= 180
        while a < -90: a += 180
        return a

print("✓ PCA Grasp Selector")

✓ PCA Grasp Selector


## 6️⃣ Initialize Components

In [18]:
gripper = SmartGripperController(port=ESP32_PORT, baudrate=ESP32_BAUDRATE)
robot = DobotControllerTCP(homography_matrix=HOMOGRAPHY_MATRIX, r_offset=ROBOT_R_OFFSET)
detector = ObjectDetector(yolo_model, PIXELS_PER_MM)
grasp_selector = PCAGraspSelector(PIXELS_PER_MM)
print("✓ Components initialized (v12)")

✓ Components initialized (v12)


---
# 📐 LIDAR OFFSET CALIBRATION (v12)
---

## วิธี Calibrate:
1. รัน Cell ด้านล่าง
2. ใช้ Dobot Studio ย้าย Robot ให้ **LIDAR ตรงกับวัตถุ** (R ต้องเป็น R_OFFSET)
3. จดตำแหน่ง Robot (X1, Y1) ที่ LIDAR ตรงวัตถุ
4. ย้าย Robot ให้ **Gripper ตรงกับวัตถุ** (R ต้องเป็น R_OFFSET เหมือนเดิม)
5. จดตำแหน่ง Robot (X2, Y2) ที่ Gripper ตรงวัตถุ
6. คำนวณ: `LIDAR_X_OFFSET = X1 - X2`, `LIDAR_Y_OFFSET = Y1 - Y2`

In [ ]:
print("="*60)
print("📐 LIDAR OFFSET CALIBRATION")
print("="*60)
print(f"\n⚠️ สำคัญ: ทุกครั้งที่ย้าย Robot ต้องใช้ R = {ROBOT_R_OFFSET}")
print("\nขั้นตอน:")
print("1. วางวัตถุบนพื้น")
print("2. ใช้ Dobot Studio ย้าย Robot ให้ LIDAR ชี้ตรงวัตถุ")
print("3. จดตำแหน่ง X1, Y1")
print("4. ย้าย Robot ให้ Gripper อยู่ตรงวัตถุ (R เดิม)")
print("5. จดตำแหน่ง X2, Y2")
print("\n" + "="*60)

# Input positions
print("\n📍 ตำแหน่ง LIDAR ตรงวัตถุ:")
x1 = float(input("   X1 = "))
y1 = float(input("   Y1 = "))

print("\n📍 ตำแหน่ง Gripper ตรงวัตถุ:")
x2 = float(input("   X2 = "))
y2 = float(input("   Y2 = "))

# Calculate offset
lidar_x_offset = x1 - x2
lidar_y_offset = y1 - y2

print("\n" + "="*60)
print("✅ CALIBRATION RESULT:")
print(f"   LIDAR_X_OFFSET = {lidar_x_offset:.2f}")
print(f"   LIDAR_Y_OFFSET = {lidar_y_offset:.2f}")
print("="*60)
print("\n📝 Copy ค่านี้ไปใส่ใน Cell Configuration (Cell 3)")

In [ ]:
# อัปเดตค่า Offset หลัง calibrate (รันหลังจาก calibrate เสร็จ)
LIDAR_X_OFFSET = 25.08   # ← ใส่ค่าที่ได้จาก calibration
LIDAR_Y_OFFSET = 20.71   # ← ใส่ค่าที่ได้จาก calibration

print(f"✅ LIDAR Offset Updated: X={LIDAR_X_OFFSET}, Y={LIDAR_Y_OFFSET}")

---
# 🤖 CONNECT ROBOT & GRIPPER
---

In [19]:
print("="*60)
print("🤖 Connecting to Robot and Gripper+LIDAR...")
print("="*60)

gripper.connect()
robot.connect(ROBOT_IP)
robot.home()

print("✅ Ready!")

🤖 Connecting to Robot and Gripper+LIDAR...
✅ Gripper+LIDAR on COM9
✅ Robot connected!
🤖 HOME...
✅ Ready!


---
# 🎯 FULL ROBOT PICK v12 (LIDAR)
---

In [20]:
selected_object = None
selected_grasp = None
current_grasps = []
detected_objects = []

def mouse_callback(event, x, y, flags, param):
    global selected_object, selected_grasp, current_grasps
    if event == cv2.EVENT_LBUTTONDOWN:
        for obj in detected_objects:
            bx,by,bw,bh = obj['bbox']
            if bx <= x <= bx+bw and by <= y <= by+bh:
                selected_object = obj
                current_grasps = grasp_selector.analyze_object(obj)
                selected_grasp = current_grasps[0] if current_grasps else None
                if selected_grasp:
                    print(f"\n📦 Object: W={selected_grasp['width_mm']:.1f}mm")
                break

def pick_with_lidar_v12(obj, grasp):
    """v12: Pick using LIDAR with calibrated offset"""
    cx, cy = grasp['center']
    grip_w = grasp['width_mm']
    camera_angle = grasp['camera_angle']
    
    # ใช้ R_OFFSET ตลอดเพื่อให้ LIDAR อยู่ตำแหน่งเดิม
    robot_r = ROBOT_R_OFFSET
    
    # ตำแหน่ง Gripper (จาก pixel)
    gripper_x, gripper_y = robot.pixel_to_robot(cx, cy)
    
    # ตำแหน่ง LIDAR (offset จาก Gripper)
    lidar_x = gripper_x + LIDAR_X_OFFSET
    lidar_y = gripper_y + LIDAR_Y_OFFSET
    
    print(f"\n🤖 Pick v12 (LIDAR): W={grip_w:.1f}mm")
    print(f"   Gripper: ({gripper_x:.1f}, {gripper_y:.1f})")
    print(f"   LIDAR:   ({lidar_x:.1f}, {lidar_y:.1f})")
    
    # 1. Safe position (หมุนไปที่ R_OFFSET ก่อน)
    print("🔄 Safe position...")
    robot.joint_move_and_wait(0, 0, 0, 0, 3)
    
    # 2. Open gripper
    gripper.open_for_object(GRIPPER_MAX_WIDTH_MM)
    time.sleep(2)
    
    # 3. Move LIDAR to above object (ให้ LIDAR ตรงวัตถุ)
    print(f"📏 Moving LIDAR above object (Z{Z_MEASURE})...")
    robot.move_to_and_wait(lidar_x, lidar_y, Z_MEASURE, robot_r, 2)
    time.sleep(2)
    
    # 4. Read LIDAR
    lidar_dist = gripper.read_lidar(samples=15)
    # # # # #
    time.sleep(4)
    if lidar_dist is None:
        print("❌ LIDAR read failed! Aborting.")
        robot.home()
        return
    
    print(f"📏 LIDAR: {lidar_dist} mm")
    
    # 5. Calculate Z_GRASP
    z_grasp = Z_MEASURE - lidar_dist + LIDAR_Z_OFFSET
    z_grasp = max(Z_FLOOR, z_grasp)
    
    print(f"📍 Z_GRASP = {Z_MEASURE} - {lidar_dist} + {LIDAR_Z_OFFSET} = {z_grasp:.1f}")
    
    # 6. Move Gripper to above object (ย้าย Gripper ไปตรงวัตถุ)
    print("🎯 Moving Gripper above object...")
    robot.move_to_and_wait(gripper_x, gripper_y, Z_MEASURE, robot_r, 2)
    
    # 7. หมุนไปมุมที่ต้องการจับ แล้วลงไปจับ
    final_r = robot.camera_angle_to_robot_r(camera_angle)
    print(f"🔄 Rotating to R={final_r:.1f}° and grasping...")
    robot.move_to_and_wait(gripper_x, gripper_y, z_grasp, final_r, 2)
    
    # 8. Grip
    gripper.grip_object(grip_w - 8.5)
    time.sleep(4)

    # 8.1 ยกขึ้น 50 mm ก่อน
    z_lift = z_grasp + 50
    print(f"⬆️ Lifting to Z={z_lift:.1f}...")
    robot.move_to_and_wait(gripper_x, gripper_y, z_lift, final_r, 2)
    
    # 9. Lift and go to drop
    robot.move_to_and_wait(*DROP_POS[:3], DROP_POS[3], 3)
    robot.move_to_and_wait(*DROP_POS[:3], DROP_POS[3], 3)
    
    # 10. Release
    gripper.release()
    time.sleep(2)
    robot.home()
    print("✅ Complete!")

def draw_grasps(frame, grasps, selected):
    for g in grasps:
        cx, cy = g['center']
        angle = g['camera_angle']
        is_sel = (selected and g == selected)
        color = (0,0,255) if is_sel else (0,255,0)
        thick = 3 if is_sel else 2
        
        length = 40
        dx = int(length * np.cos(np.radians(angle)))
        dy = int(length * np.sin(np.radians(angle)))
        cv2.line(frame, (cx-dx, cy-dy), (cx+dx, cy+dy), color, thick)
        cv2.circle(frame, (cx, cy), 5, color, -1)

# Main loop
cap = cv2.VideoCapture(CAMERA_ID)
cv2.namedWindow('Pick v12 LIDAR')
cv2.setMouseCallback('Pick v12 LIDAR', mouse_callback)

print("="*60)
print("🎯 PICK v12 (LIDAR with Calibrated Offset)")
print("Click=Select | SPACE=Execute | H=Home | Q=Quit")
print("="*60)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    
    detected_objects = detector.detect(frame)
    
    display = frame.copy()
    for obj in detected_objects:
        x,y,w,h = obj['bbox']
        is_sel = (selected_object and obj['center'] == selected_object['center'])
        cv2.rectangle(display, (x,y), (x+w,y+h), (0,0,255) if is_sel else (0,255,0), 2)
    
    if selected_object and current_grasps:
        draw_grasps(display, current_grasps, selected_grasp)
    
    cv2.rectangle(display, (0,0), (640,35), (30,30,30), -1)
    cv2.putText(display, f"v12 LIDAR | Obj:{len(detected_objects)} | SPACE | H | Q",
               (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255,255,255), 1)
    
    if selected_grasp:
        cv2.putText(display, f"[W={selected_grasp['width_mm']:.1f}mm - SPACE to pick]",
                   (10, 470), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)
    
    cv2.imshow('Pick v12 LIDAR', display)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'): break
    elif key == ord('r'):
        selected_object = None
        selected_grasp = None
        current_grasps = []
    elif key == ord('h'):
        robot.home()
    elif key == ord(' ') and selected_object and selected_grasp:
        pick_with_lidar_v12(selected_object, selected_grasp)
        selected_object = None
        selected_grasp = None
        current_grasps = []

cap.release()
cv2.destroyAllWindows()
gripper.disconnect()

🎯 PICK v12 (LIDAR with Calibrated Offset)
Click=Select | SPACE=Execute | H=Home | Q=Quit
